# Hyper parameter tunning
- updated 0917
- data  
5th_FE_ver4_before   
5th_FE_ver4  

- ML basic 모델들 튜닝

## y_predict 값들 피클로 저장해놓기
pred_list, test_list, idx


## ready

In [ ]:
cd /content/drive/My Drive/빅콘

/content/drive/My Drive/빅콘


In [ ]:
!apt-get update --qq
!apt-get install fonts-nanum* -qq

E: Command line option --qq is not understood in combination with the other options


In [ ]:
# 한글 폰트
import matplotlib.font_manager as fm
fm._rebuild()

In [ ]:
# # CUDA
# !pip install mxnet-cu80
# import mxnet as mx

In [ ]:
# ready
import os
import sys
import joblib
import pickle
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd 
import numpy as np
import random
import math

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

import imp
import datetime
from tqdm import tqdm

In [ ]:
# vusualize
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

In [ ]:
# sklearn
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score

## Load Data
- 5th_FE_ver4_before  
- 5th_FE_ver4


data['X'] / data['y']

### 5th_FE_ver4

In [ ]:
# load data
## ver3
v4 = joblib.load(os.path.join('data', '5th_FE_ver4.pkl'))
locals().update(v4)

v4_label = joblib.load(os.path.join('data', '5th_FE_ver4_before.pkl'))
locals().update(v4_label)

In [ ]:
# X, y 설정
X = v4['X'] ; y = v4['y'] ; log_y = np.log(y)
X_label = v4_label['X']

In [ ]:
X.isnull().sum().sum(), X_label.isnull().sum().sum()

(267028, 267028)

In [ ]:
# fill na
X = X.fillna(0); X_label = X_label.fillna(0)

# One Hot
X_dum = pd.get_dummies(X)
X.shape, X_dum.shape, X_label.shape

((35379, 334), (35379, 334), (35379, 276))

### Cross Validation function

In [ ]:
# mape
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# mape with exp
def MAPE_exp(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

In [ ]:
# MAPE_exp_cv 구현
def MAPE_exp_cv5(model,X,y,cv_splits):        
    # setting
    mape = []
    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=42,shuffle=True)
    for t,v in cv.split(X):
        X_train=X.iloc[t]       # 훈련용
        X_val=X.iloc[v]         # 검증용 분리.
        y_train=y.iloc[t]
        y_val=y.iloc[v]

        # model
        model.fit(X_train,y_train)
        y_pred = model.predict(X_val)
        #y_pred_list.append(np.exp(y_pred))
        # mape
        mape.append(MAPE_exp(y_val, y_pred))
    return mape, np.mean(mape), y_pred_list

In [ ]:
# stacking with cv=5
# MAPE_exp_cv 구현

def MAPE_exp_cv5_st(model,X,y,cv_splits,version):        
    # setting
    mape = {'val_mape' : [], 'test_mape' : [],'final_mape' : []}
    pred = {'val_idx' : [], 'val_pred' : [],
            'test_idx' : [], 'test_pred' : [],
            'final_pred' : []}      # final : test set mean 값
    
    # index save
    val_idx = []
    test_idx = []

    # split train, test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=77)
    pred['test_idx'].append(X_test.index)

    # for 문 돌리기
    cv=KFold(n_splits=cv_splits,random_state=77,shuffle=True)
    for t,v in cv.split(X_train):
        X_train_ = X_train.iloc[t]       # 훈련용
        X_val_ = X_train.iloc[v]         # 검증용 분리.
        y_train_ = y_train.iloc[t]
        y_val_ = y_train.iloc[v]

        pred['val_idx'].append(v)

        # model
        model.fit(X_train_,y_train_)
        y_pred = model.predict(X_val_)
        pred['val_pred'].append(np.exp(y_pred))

        test_pred = model.predict(X_test)
        pred['test_pred'].append(np.exp(test_pred))

        # mape
        mape['val_mape'].append(MAPE_exp(y_val_, y_pred))
        mape['test_mape'].append(np.exp(test_pred))

    # final values
    final_test = np.mean(pred['test_pred'], axis=0)
    final_mape = MAPE_exp(y_test, final_test)
    mape['final_mape'].append(final_mape)
    pred['final_pred'].append(final_test)

    # save pickle
    with open('mape.pickle' + version, 'wb') as f:
        pickle.dump(mape, f, pickle.HIGHEST_PROTOCOL)
    with open('pred.pickle' + version, 'wb') as f:
        pickle.dump(pred, f, pickle.HIGHEST_PROTOCOL)

    '''
    # load
    with open('data.pickle', 'rb') as f:
    data = pickle.load(f)
    '''

    return mape, pred

# tunning

### ML + gridsearch
label data 에서 Bad value for num_feature 이슈때문에 X_dum 사용  


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

* knn + gridsearch

In [ ]:
# Setting X, y
X = X_dum
y = log_y

# model
knn = KNeighborsRegressor()

# params
params = {
    'algorithm' : ['auto','ball_tree','kd_tree','brute'],
    'n_neighbors': range(3,30,2) # usually odd numbers
}

# grid_search
grid_search = GridSearchCV(estimator=knn,
                           param_grid=params,
                           cv=5,
                           verbose=2) # Turn on cv train scores
grid_search.fit(X, y)


Fitting 5 folds for each of 56 candidates, totalling 280 fits
[CV] algorithm=auto, n_neighbors=3 ...................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .................... algorithm=auto, n_neighbors=3, total=   6.9s
[CV] algorithm=auto, n_neighbors=3 ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.9s remaining:    0.0s


[CV] .................... algorithm=auto, n_neighbors=3, total=   5.6s
[CV] algorithm=auto, n_neighbors=3 ...................................
[CV] .................... algorithm=auto, n_neighbors=3, total=   7.6s
[CV] algorithm=auto, n_neighbors=3 ...................................
[CV] .................... algorithm=auto, n_neighbors=3, total=   6.3s
[CV] algorithm=auto, n_neighbors=3 ...................................
[CV] .................... algorithm=auto, n_neighbors=3, total=   5.0s
[CV] algorithm=auto, n_neighbors=5 ...................................
[CV] .................... algorithm=auto, n_neighbors=5, total=   6.9s
[CV] algorithm=auto, n_neighbors=5 ...................................
[CV] .................... algorithm=auto, n_neighbors=5, total=   5.7s
[CV] algorithm=auto, n_neighbors=5 ...................................
[CV] .................... algorithm=auto, n_neighbors=5, total=   7.7s
[CV] algorithm=auto, n_neighbors=5 ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 280 out of 280 | elapsed: 30.2min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': range(3, 30, 2)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [ ]:
# best params
grid_search.best_params_

{'algorithm': 'ball_tree', 'n_neighbors': 17}

In [ ]:
# best knn modeling
best_knn = KNeighborsRegressor(algorithm='ball_tree',
                          n_neighbors=17)

In [ ]:
# save
knn_mepe0917, knn_pred0917 = MAPE_exp_cv5_st(best_knn,X_dum,log_y,5,'knn_0917')

In [ ]:
knn_mepe0917, knn_pred0917

({'final_mape': [inf],
  'test_mape': [array([ 3645173.166989  , 15613041.93804391, 25352235.94602551, ...,
          37098751.92143043, 14664188.56588906, 31750204.22732116]),
   array([ 3755548.14941166, 18178142.50409776, 24930775.71468945, ...,
          38912060.11445337, 16870877.7484387 , 35280901.74755433]),
   array([ 3271348.41851496, 19441141.94773366, 22577277.0609805 , ...,
          34126935.29929838, 12929177.74168317, 35011573.20187232]),
   array([ 3094984.10448428, 15158594.86241597, 26342092.64592642, ...,
          38319897.8347826 , 14076052.52428267, 30620627.66187099]),
   array([ 3644797.11739154, 17462495.67601562, 26706171.67014952, ...,
          29837487.18064287, 14048880.07615303, 31157433.74060576])],
  'val_mape': [74.57599736350421,
   76.4837947933578,
   76.09613915809452,
   76.72493020783352,
   75.2988743324274]},
 {'final_pred': [array([ 3482370.19135829, 17170683.38566138, 25181710.60755428, ...,
          35659026.47012153, 14517835.33128933, 32